In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import utils as ut
import yaml
import json

## Reading

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)
with open(Path('conf') / 'product_list.yaml') as file:
    product_cat = yaml.full_load(file)
with open(Path('conf') / 'payment_list.yaml') as file:
    payment_cat = yaml.full_load(file)

data         = pd.read_json(Path(config['input_path']) / 'db_mesinfo.json', orient='index').fillna('')
locations_df = pd.read_csv(Path(config['input_path']) / 'municipis_merge.csv').fillna('')

print(data.shape)

data.rename(columns={'url':'URL', 
                     'Nom de la persona productora:':'PRODUCTOR', 
                     'Marca:':'MARCA', 
                     'Municipi:':'MUNICIPIO',  
                     'On serveix:':'DONDE', 
                     'Productes disponibles:':'PRODUCTOS', 
                     'Altres productes alimentaris:':'OTROS',
                     'Possibilitats pagament:': 'PAGO', 
                     'Fruita*:':'FRUTA', 
                     '*':'NOTAS', 
                     'Més informació:':'INFO', 
                     'Carn:':'CARNE', 
                     'Verdura*:': 'VERDURA', 
                     'Flor i planta ornamental:':'FLORES'}, inplace=True)

data.head()

(374, 14)


,URL,PRODUCTOR,MARCA,MUNICIPIO,DONDE,PRODUCTOS,OTROS,PAGO,INFO,VERDURA,NOTAS,FRUTA,CARNE,FLORES
piteus-vins-vermuts-i-cava,https://pagesiaacasa.cat/productor/piteus-vins...,"PITEUS vins, vermuts i cava",PITEUS,Corbera del Llobregat (Baix Llobregat),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Vi,"Efectiu, Transferència prèvia, Targeta",,,,,,
jordi-riera-flores,https://pagesiaacasa.cat/productor/jordi-riera...,Jordi Riera Flores,Molí de Rué,Vinebre (Ribera d’Ebre),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,"Oli, Vi i caves","Efectiu, Bizum, Transferència prèvia, Targeta",<p>Vins DOP Tarragona</p>,,,,,
agusti-segranyes-i-carulla,https://pagesiaacasa.cat/productor/agusti-segr...,Agustí Segranyes i Carulla (Producció ecològica),La Cabana d'Alcover SL,Ponts (Noguera),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Transferència prèvia",,,,,,
pilar-comes,https://pagesiaacasa.cat/productor/pilar-comes,Pilar Comes (Producció ecològica),PARC DE LES OLORS DEL SERRAT,SANTA EULÀLIA DE RONÇANA (Vallès Oriental),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Herbes Aromàtiques,Transferència prèvia,,"Bledes, Calçots, Carxofes, Enciams, Escaroles,...",*Disponibilitat segons temporada,,,
sergi-martin-garcia,https://pagesiaacasa.cat/productor/sergi-marti...,Sergi Martín García (Producció integrada),COLLITA DE L'AVI PIÀ,Valls (Alt Camp),"Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell...",,Oli,"Efectiu, Bizum, Transferència prèvia, Targeta",,"Cebes, Tomàquets",*Disponibilitat segons temporada,,,


In [3]:
stopwords = ut.get_all_stopwords()

## Pre-process columns

In [4]:
ut.run_preprocess_on_cols(data,['OTROS', 'PAGO'],stopwords)

## New columns

### Binary varibles

In [5]:
# create binary variables representing whether they have a payment method or not
ut.create_binary_var(data,payment_cat,'PAGO'+'_prep')
# create binary variables representing whether they have a type of product or not
ut.create_binary_var(data,product_cat,'OTROS'+'_prep')

## Updating the columns vegetables, fruit, meat and flowers, 
## since in this dataset are informed in different fields (and not in the main one OTROS)
data.loc[data.VERDURA != '', 'vegetables']=1
data.loc[data.FRUTA != '', 'fruit']=1
data.loc[data.CARNE != '', 'meat']=1
data.loc[data.FLORES != '', 'flowers']=1

In [6]:
#data.columns
#data[['OTROS','VERDURA', 'CARNE', 'FRUTA','vegetables','meat','fruit','charcuterie','fruit_veggies_products', 'bread_pastries', 'alcohol']].head(10)

### Numerical cols

In [7]:
data = ut.add_numerical_cols(data)

## 42 comarcas en total:   
    
#Alt Camp, Alt Empordà, Alt Penedès, Alt Urgell, Alta Ribagorça, 
#Anoia, Bages,Baix Camp, Baix Ebre, Baix Empordà,
#Baix Llobregat, Baix Penedès, Barcelonès, Berguedà, Cerdanya, 
#Conca de Barberà, Garraf, Garrigues, Garrotxa, Gironès,
#Maresme, Moianès, Montsià, Noguera, Osona / Lluçanès, 
#Pallars Jussà, Pallars Sobirà, Pla d'Urgell, Pla de l'Estany, Priorat, 
#Ribera d'Ebre, Ripollès, Segarra, Segrià, Selva,
#Solsonès, Tarragonès, Terra Alta, Urgell, Vall d'Aran, 
#Vallès Occidental,  Vallès Oriental

##NB: Hay algunos casos donde hay 41 comarcas, más tarde los consideremos como toda cataluña

### Comarca of origin (i.e. of the producer)

In [8]:
# Dictionary to translate municipis to comarca
mun_to_com_dict = locations_df[locations_df['Municipi']!=''].set_index('Municipi')['Comarca'].to_dict()

In [9]:
data['comarca_origin'] = data['MUNICIPIO'].str.split(')').str.get(-2).str.split('(').str.get(1).fillna('')

data['comarca_origin'] = data['comarca_origin'].apply(lambda x: ut.check_comarca_spelling(
    x,locations_df['Comarca'],stopwords) if x not in locations_df['Comarca'] else x)

Not found: 


In [10]:
#data.loc[data.CARNE.isna(), ['PRODUCTOR', 'MUNICIPIO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]
#data.loc[data.CARNE.isna() & data.FRUTA.isna() & data.VERDURA.isna() & data.OTROS.isna(), ['PRODUCTOR', 'MUNICIPIO','INFO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]
##7 prodctores no tienen ni carne, ni fruta, ni verdura ni otros productos

#data.loc[data.PRODUCTOR.str.contains('ANA GAYOSO')] #venden shampoo etc. (lo dice en INFO_EXTRA)
#data[data.INFO.notna()].shape ## 177 (197 no tienen info)
##data.loc[data.INFO.notna(), ['PRODUCTOR', 'MUNICIPIO','INFO','CARNE', 'FRUTA', 'VERDURA', 'OTROS']]

#data.INFO.unique()
#data.groupby(['INFO']).agg({'PRODUCTOR': ['count']})
#data.INFO.head(10)

In [11]:
data.drop(['PAGO_prep','OTROS_prep'],axis=1).to_csv(Path(config['input_path']) / 'pagesos_clean.csv', index=False)
##data.to_csv(Path(config['input_path']) / 'pagesos_clean.csv', index=False)
#print(data.columns)